## Índice

1. [Introducción y objetivos](#1-introducción-y-objetivos)
2. [Carga de datos y preparación](#2-carga-de-datos-y-preparación)
3. [Extracción y limpieza de texto](#3-extracción-y-limpieza-de-texto)
    - 3.1. Ejemplo de estructura del PDF
    - 3.2. Detección y eliminación de ruido
    - 3.3. Análisis de títulos y secciones
4. [Generación del corpus estructurado](#4-generación-del-corpus-estructurado)
5. [Notas y próximos pasos](#5-notas-y-próximos-pasos)

---

## 1. Introducción y objetivos

En este notebook se va a extraer y limpiar texto de documentos normativos en PDF para montar un corpus usable en tareas de recuperación de información. El objetivo es dejar los datos listos para probar chunking y modelos de RAG sobre casos reales, no solo datasets de ejemplo.

---

## 2. Carga de datos y preparación

Se cargan los PDFs en bruto desde la carpeta del proyecto.

In [1]:
import os

pdf_dir = "../data/raw_pdfs"
pdf_files = [f for f in os.listdir(pdf_dir) if f.lower().endswith(".pdf")]
pdf_files

['ai_hleg_ethics_guidelines.pdf',
 'eu_ai_act_regulation.pdf',
 'nist_privacy_framework_v1.pdf',
 'oecd_ai_classification_framework.pdf',
 'oecd_legal_0449_en.pdf']

## 3. Extracción y limpieza de texto

Se empieza la extracción del texto de los PDFs para ver la estructura y el tipo de contenido que tiene cada uno. El objetivo aquí es detectar qué ruido hay (como encabezados, pies de página o tablas raras) y ver si los títulos y secciones se pueden distinguir bien.

De manera manual, se han revisado por encima los PDFs y se quitan las sigiuentes páginas:

| PDF                           | Páginas útiles       |
|-------------------------------|---------------------|
| oecd_legal_0449_en.pdf        | 3 a 9               | 
| eu_ai_act_regulation.pdf      | 2 a 95              | 
| oecd_ai_classification_framework.pdf | 16 a 67       |
| ai_hleg_ethics_guidelines.pdf | 4 a 37              |
| nist_privacy_framework_v1.pdf | 5 a 43              |


In [3]:
import fitz  # PyMuPDF

# zero-based index
pages_dict = {
    "oecd_legal_0449_en.pdf": range(2, 9),
    "eu_ai_act_regulation.pdf": range(1, 95),
    "oecd_ai_classification_framework.pdf": range(15, 67),
    "ai_hleg_ethics_guidelines.pdf": range(3, 37),
    "nist_privacy_framework_v1.pdf": range(4, 43),
}

for pdf_file in pdf_files:
    if pdf_file not in pages_dict:
        continue
    pdf_path = os.path.join(pdf_dir, pdf_file)
    doc = fitz.open(pdf_path)
    page_idxs = list(pages_dict[pdf_file])
    print(f"\n--- {pdf_file} ({len(page_idxs)} páginas útiles) ---")

    for idx, pagina_idx in enumerate([0, 2]):
        if pagina_idx >= len(page_idxs):
            continue
        p = page_idxs[pagina_idx]
        page = doc.load_page(p)
        print(f"\n--- Página real {p+1} (índice útil {pagina_idx+1}) ---")
        print(page.get_text())



--- ai_hleg_ethics_guidelines.pdf (34 páginas útiles) ---

--- Página real 4 (índice útil 1) ---
2 
 
EXECUTIVE SUMMARY 
The aim of the Guidelines is to promote Trustworthy AI. Trustworthy AI has three components, which should be 
met throughout the system's entire life cycle: (1) it should be lawful, complying with all applicable laws and 
regulations (2) it should be ethical, ensuring adherence to ethical principles and values and (3) it should be robust, 
both from a technical and social perspective since, even with good intentions, AI systems can cause unintentional 
harm. Each component in itself is necessary but not sufficient for the achievement of Trustworthy AI. Ideally, all 
three components work in harmony and overlap in their operation. If, in practice, tensions arise between these 
components, society should endeavour to align them.  
These Guidelines set out a framework for achieving Trustworthy AI. The framework does not explicitly deal with 
Trustworthy AI’s first comp